In [1]:
import numpy as np
import cv2
import scipy

### 1)

In [2]:
source = np.load("./short_questions/1a.npy").astype(np.float32)
target = np.load("./short_questions/1b.npy").astype(np.float32)

source = source / np.reshape(source[:,2], (10,1))
target = target / np.reshape(target[:,2], (10,1))

# print(source)
# print(target)

In [3]:
A = np.zeros((20, 6))
b = np.zeros((20, 1))

j = 0

for i in range(10):
  s = source[i]
  t = target[i]

  A[j]   = np.array([s[0], s[1], 1, 0, 0, 0])
  A[j+1] = np.array([0, 0, 0, s[0], s[1], 1])

  b[j]   = t[0]
  b[j+1] = t[1]

  j += 2

t = np.linalg.lstsq(A, b)[0]
transform = np.zeros((3, 3))
transform[:2] = np.reshape(t, (2,3))
transform[2] = np.array([0, 0, 1])

print(transform)

[[1.9999998  2.99999981 7.0000001 ]
 [1.49999982 0.99999993 8.70000008]
 [0.         0.         1.        ]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


### 2)

In [4]:
source = np.load("./short_questions/2a.npy").astype(np.float32)
target = np.load("./short_questions/2b.npy").astype(np.float32)

source = source / np.reshape(source[:,2], (10,1))
target = target / np.reshape(target[:,2], (10,1))

# print(source)
# print(target)

In [11]:
A = np.zeros((20, 9))

j = 0

for i in range(10):
  s = source[i]
  t = target[i]
  A[j]   = np.array([s[0], s[1], 1, 0, 0, 0, -t[0] * s[0], -t[0] * s[1], -t[0]])
  A[j+1] = np.array([0, 0, 0, s[0], s[1], 1, -t[1] * s[0], -t[1] * s[1], -t[1]])
  j += 2


eigenValues, eigenVectors = np.linalg.eig(A.T @ A)

idx = eigenValues.argsort()  
eigenValues = eigenValues[idx]
eigenVectors = eigenVectors[:,idx]

v = eigenVectors[:, 0]
v = v.reshape((3,3))
v /= v[2,2]

print(v)

[[ -5.34103206   2.90240039   6.72625209]
 [-13.24281031   1.7102528   11.05260512]
 [ -0.87295407  -0.1056948    1.        ]]


### 3)

In [6]:
source = np.load("./short_questions/3a.npy").astype(np.float32)
target = np.load("./short_questions/3b.npy").astype(np.float32)

source = source / np.reshape(source[:,2], (15,1))
target = target / np.reshape(target[:,2], (15,1))

# print(source)
# print(target)

In [7]:
def get_homography(pts1, pts2):
  pts1 = pts1.astype(np.float32)
  pts2 = pts2.astype(np.float32)

  num_pts = pts1.shape[0]
  A = np.zeros((num_pts * 2, 9))
  j = 0

  for i in range(num_pts):
    s = pts1[i]
    t = pts2[i]
    A[j]   = np.array([s[0], s[1], 1, 0, 0, 0, -t[0] * s[0], -t[0] * s[1], -t[0]])
    A[j+1] = np.array([0, 0, 0, s[0], s[1], 1, -t[1] * s[0], -t[1] * s[1], -t[1]])
    j += 2

  eigenValues, eigenVectors = np.linalg.eig(A.T @ A)

  idx = eigenValues.argsort()  
  eigenValues = eigenValues[idx]
  eigenVectors = eigenVectors[:,idx]

  v = eigenVectors[:, 0]
  v = v.reshape((3,3))
  v = v / v[2,2]

  return v



def get_best_tranform(source, target):
  # Some parameters
  num_samples = 4
  num_trials = 500
  total_matches = 15
  inlier_thresh = 1

  # To keep track of the best transformation
  xform = np.zeros((3, 3))
  most_inliers = 0

  # Loop through num_trials times
  for i in range(num_trials):

    # Randomly choose num_samples indices from total number of matches
    choices = np.random.choice(total_matches, num_samples, replace=False)

    # Get the matching keypoint coordinates from those indices
    pts1 = source[choices].astype(np.float32)
    pts2 = target[choices].astype(np.float32)

    # Get the transformation matrix of the homography
    transformation = get_homography(pts1, pts2)

    num_inliers = 0

    # Loop through all of our matches
    for j in range(total_matches):

      # Get the coordinates of the matching features using get_keypoint_coord_from_match
      p1 = source[j]
      p2 = target[j]

      # Compute the expected coordinate after transforming the keypoint from img1
      # using the computed affine transformation
      expect = transformation @ np.float32([p1[0], p1[1], 1])

      # If the distance between the expected coordinate and the actual coordinate in im2
      # is less than the inlier threshold, increment the number of inliers
      dist = np.sqrt((expect[0] - p2[0])**2 + (expect[1] - p2[1])**2)
      if dist < inlier_thresh:
        num_inliers = num_inliers + 1


    # If for this transformation we have found the most inliers update most_inliers and xform
    if num_inliers > most_inliers:
      most_inliers = num_inliers
      xform = np.copy(transformation)
  
  return xform

In [8]:
transform = get_best_tranform(source, target)
print(transform)

[[2.00000011e+00 6.00000044e+00 6.00000029e+00]
 [4.00000015e+00 5.00000002e+00 3.00000006e+00]
 [1.49703243e-08 7.07211657e-10 1.00000000e+00]]


### 4)

In [9]:
grads = np.load("./short_questions/gradients.npy")
mags = np.sqrt(grads[:,0]**2 + grads[:,1]**2)
angs = np.arctan2(grads[:,1], grads[:,0])
angs[angs<0] += 2 * np.pi

hist, _ = np.histogram(angs, 10, weights=mags)
print(hist)

[ 9.14053171  5.39536657  4.13864855 13.50065006  8.05100898  3.12836609
  1.86109347  1.68195599 12.44989872 11.57830046]


### 5)

In [10]:
import matplotlib.pyplot as plt

img1 = cv2.imread("./data/bikes1.png")
img2 = cv2.imread("./data/graf1.png")
img3 = cv2.imread("./data/leuven1.png")
img4 = cv2.imread("./data/wall1.png")

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY).astype('float32')
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY).astype('float32')
gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY).astype('float32')
gray4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY).astype('float32')


imgs = [gray1, gray2, gray3, gray4]
hists = []

for i in imgs:
  Gx,Gy = np.gradient(i)

  mags = np.sqrt(Gx**2 + Gy**2).astype(np.float32)

  angs = np.arctan2(Gy, Gx)
  angs[angs<0] += 2 * np.pi

  hist, _ = np.histogram(angs, 10, weights=mags)
  hists.append(hist)
  

j = 0
for h in hists:
  print("Image ", j, h)
  j = j+1

Image  0 [811916.44 380523.72 598272.3  332306.56 411495.1  766878.44 383158.34
 618075.7  326856.78 464770.25]
Image  1 [207928.5  247302.3  171725.44 237208.88 189970.27 214890.53 248541.03
 173782.31 237119.4  194507.16]
Image  2 [830519.75 440489.97 603673.9  346149.22 477970.3  902450.44 425198.06
 575750.6  336104.44 457988.6 ]
Image  3 [1053983.5   681122.6   981762.75 1374315.9  1454261.5   999179.3
  714946.6  1048856.   1300525.6  1472427.8 ]
